In [1]:
%load_ext autoreload
import numpy as np
import plotly.graph_objects as go

%autoreload 2
from zcu_tools.notebook.persistance import load_result
from zcu_tools.simulate.fluxonium import (
    calculate_system_n_oper_vs_flx,
    calculate_n_oper_vs_flx,
    calculate_energy_vs_flx,
)

In [2]:
qub_name = "Test049"

server_ip = "005-writeboard"
port = 4999

# Load data

## Parameters

In [3]:
loadpath = f"../../result/{qub_name}/params.json"
_, params, _, _, _, result = load_result(loadpath)

r_f = result["dispersive"]["r_f"]
g = result["dispersive"]["g"]

# Matrix elements

In [4]:
sweep = None
spectrum_data = None

return_dim = 4
flxs = np.linspace(0.0, 0.5, 200)

In [ ]:
spectrum_data, energies = calculate_energy_vs_flx(
    params, flxs, spectrum_data=spectrum_data
)

In [ ]:
spectrum_data, elements = calculate_n_oper_vs_flx(
    params, flxs, return_dim=return_dim, spectrum_data=spectrum_data
)

In [ ]:
sweep, side_elements = calculate_system_n_oper_vs_flx(
    params, flxs, r_f, g=g, return_dim=return_dim, sweep=sweep
)

In [8]:
bypass_levels = [None, 2, 3]

In [ ]:
fig = go.Figure()

for f, t in [(0, 1), (1, 0)]:
    for bypass_level in bypass_levels:
        if bypass_level is None:
            m = np.abs(side_elements[:, f, t])
            label = f"{f}->{t}"
            energy = energies[:, t] - energies[:, f]
            m[energy > r_f] = 0
        else:
            m1 = np.abs(elements[:, f, bypass_level])
            m2 = np.abs(side_elements[:, bypass_level, t])
            m = np.sqrt(m1 * m2)
            label = f"{f}->{bypass_level}->{t}"
            energy1 = energies[:, bypass_level] - energies[:, f]
            energy2 = energies[:, t] - energies[:, bypass_level]
            m[np.logical_or(energy1 < 0, energy2 > r_f)] = 0
        fig.add_trace(go.Scatter(x=flxs, y=m, mode="lines", name=label))

fig.update_layout(
    yaxis_range=[0, 0.2],
    xaxis_title="Flux",
    yaxis_title="Matrix Elements",
    showlegend=True,
)
fig.show()

In [10]:
prefix = f"../../result/{qub_name}"
fig.write_html(f"{prefix}/web/reset_element.html", include_plotlyjs="cdn")
fig.write_image(
    f"{prefix}/image/reset_element.png", format="png", width=800, height=400
)